# Model input ERP format preparation 

## Imports

In [1]:
import mne      # toolbox for analyzing and visualizing EEG data
import os       # using operating system dependent functionality (folders)
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import copy     # Can Copy and Deepcopy files so original file is untouched.
from ipywidgets import IntSlider, Output
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from math import nan

import sys
sys.path.insert(0, '../eegyolk') # path to helper functions
import helper_functions as hf # library useful for eeg and erp data cleaning
import initialization_functions #library to import data
import epod_helper

In [2]:
metadata = pd.read_csv('metadata.csv', sep = ',')

In [3]:
metadata

,eeg_file,ParticipantID,test,sex,age_months,age_months_days,dyslexic_parent,Group_AccToParents,eeg_path,path_epoch,epoch_file
0,101a,101,a,m,20,20;22,m,At risk,F:\Stage\ePODIUM\Data\ePodium (Projectfolder)\...,F:\Stage\ePODIUM\ePodium\dyslexiaprediction_na...,101a_epo.fif
1,101b,101,b,m,23,23;16,m,At risk,F:\Stage\ePODIUM\Data\ePodium (Projectfolder)\...,F:\Stage\ePODIUM\ePodium\dyslexiaprediction_na...,101b_epo.fif
2,102a,102,a,f,20,20;27,Nee,Control,F:\Stage\ePODIUM\Data\ePodium (Projectfolder)\...,F:\Stage\ePODIUM\ePodium\dyslexiaprediction_na...,102a_epo.fif
3,102b,102,b,f,23,23;16,Nee,Control,F:\Stage\ePODIUM\Data\ePodium (Projectfolder)\...,F:\Stage\ePODIUM\ePodium\dyslexiaprediction_na...,102b_epo.fif
4,103a,103,a,f,20,20;23,m,At risk,F:\Stage\ePODIUM\Data\ePodium (Projectfolder)\...,F:\Stage\ePODIUM\ePodium\dyslexiaprediction_na...,103a_epo.fif


## Read all epochs from files

In [4]:
dataframe = metadata
epochs = []
for index, file in dataframe.iterrows():
    print(f"Checking out file: {file['epoch_file']}")
    path = os.path.join(file['path_epoch'], file['epoch_file'])
    epoch = mne.read_epochs(path, preload=False)
    epochs.append(epoch)

Checking out file: 101a_epo.fif
Reading F:\Stage\ePODIUM\ePodium\dyslexiaprediction_nadine\epochs\101a_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2266 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 101b_epo.fif
Reading F:\Stage\ePODIUM\ePodium\dyslexiaprediction_nadine\epochs\101b_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2440 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 102a_epo.fif
Reading F:\Stage\ePODIUM\ePodium\dyslexiaprediction_nadine\epochs\102a_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2389 matching events found
No baseline correction applied
0 p

## Create pandas dataframe with the average difference between standard and deviant responses

In [5]:
# define the events for standard and deviant
standard_events = [2,5,8,11]
deviant_events = [3,6,9,12]

# create dataframe with expected columns 
df = pd.DataFrame(columns=["eeg_file", "paradigm", "channel", "mean"])

# loop over all participants
for i in range(len(metadata['eeg_file'])):
    
    # loop over every paradigmn per participant 
    for j in standard_events: 
        paradigm = j
        # select the standard and deviant for a specific sequence and calculate the evoked response
        std_evoked = epochs[i][j].average() 
        dev_evoked = epochs[i][j+1].average()
        
        # calculate the mismatch response between standard and deviant evoked
        evoked_diff = mne.combine_evoked([std_evoked, dev_evoked], weights=[1, -1])
        
        # get a list of all channels
        chnames_list = evoked_diff.info['ch_names']
        
        # compute for every channel the distance mean of the mismatch line
        for channel in chnames_list: 
            chnames = mne.pick_channels(evoked_diff.info['ch_names'], include=[channel])
            roi_dict = dict(left_ROI=chnames) # unfortunately combine_channels only takes a dictionary as input
            roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
            timeseries = roi_evoked.to_data_frame()
            timeseries_avg = timeseries['left_ROI'].mean()
            df = df.append({'eeg_file': metadata['eeg_file'][i], 'paradigm' : paradigm, 'channel': channel, 'mean' :  timeseries_avg}, ignore_index=True) 

df

<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoke

<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoke

<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoke

<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoke

<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoke

...

<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoke

<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoke

<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoke

<ipython-input-5-4344076d7b4a>:28: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001662B5A9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')


,eeg_file,paradigm,channel,mean
0,101a,2,Fp1,3.981499
1,101a,2,AF3,11.639081
2,101a,2,F7,-18.811622
3,101a,2,F3,18.343454
4,101a,2,FC1,10.023632
...,...,...,...,...
635,103a,11,F8,-59.793993
636,103a,11,AF4,-69.507558
637,103a,11,Fp2,-66.936390
638,103a,11,Fz,-66.624167


## Transpose dataframe into combination of paradigm and channel per participant

In [6]:
# transformation of the dataframe
df = df.pivot(index='eeg_file', columns=['paradigm','channel'])
df.columns = ['_'.join(str(s).strip() for s in col if s) for col in df.columns]
df.reset_index(inplace=True)

In [7]:
df.columns = df.columns.str[5:]
df = df.rename(columns={'ile':'eeg_file'})

In [8]:
df

,eeg_file,2_Fp1,2_AF3,2_F7,2_F3,2_FC1,2_FC5,2_T7,2_C3,2_CP1,...,11_C4,11_T8,11_FC6,11_FC2,11_F4,11_F8,11_AF4,11_Fp2,11_Fz,11_Cz
0,101a,3.981499,11.639081,-18.811622,18.343454,10.023632,-4.024143,-13.999364,-17.671889,-29.638338,...,-5.419709,-20.527842,-7.739499,-24.687588,-40.535996,-6.189181,-28.356975,-19.061485,-40.427852,-43.257143
1,101b,-9.964857,-7.810340,19.733811,3.661548,14.292444,32.492531,34.867429,41.204902,33.612812,...,-3.435515,1.738558,8.090447,13.989412,8.420473,23.518492,50.627080,33.244638,28.191358,40.034364
2,102a,2.390371,-17.843979,-3.338771,7.536740,-10.998419,18.220947,16.625307,-4.156407,-17.935160,...,26.739956,3.039871,15.349378,31.937126,41.415109,14.678230,22.992495,65.963604,31.134918,18.108797
3,102b,7.074118,23.187111,-9.459183,25.835532,18.345802,34.434200,34.885866,22.527005,37.353435,...,15.112951,36.641937,37.896288,19.602109,14.555350,41.145307,24.244979,58.820555,11.613820,9.621704
4,103a,-18.065883,-17.785199,-12.858838,-8.709094,9.764843,-1.953743,-2.150446,11.090374,0.289517,...,-39.471842,-45.274814,-41.629402,-50.573104,-69.076174,-59.793993,-69.507558,-66.936390,-66.624167,-50.016909


## Merge and safe dataframe

In [9]:
df = pd.merge(df, metadata, on='eeg_file')

In [10]:
df

,eeg_file,2_Fp1,2_AF3,2_F7,2_F3,2_FC1,2_FC5,2_T7,2_C3,2_CP1,...,ParticipantID,test,sex,age_months,age_months_days,dyslexic_parent,Group_AccToParents,eeg_path,path_epoch,epoch_file
0,101a,3.981499,11.639081,-18.811622,18.343454,10.023632,-4.024143,-13.999364,-17.671889,-29.638338,...,101,a,m,20,20;22,m,At risk,F:\Stage\ePODIUM\Data\ePodium (Projectfolder)\...,F:\Stage\ePODIUM\ePodium\dyslexiaprediction_na...,101a_epo.fif
1,101b,-9.964857,-7.810340,19.733811,3.661548,14.292444,32.492531,34.867429,41.204902,33.612812,...,101,b,m,23,23;16,m,At risk,F:\Stage\ePODIUM\Data\ePodium (Projectfolder)\...,F:\Stage\ePODIUM\ePodium\dyslexiaprediction_na...,101b_epo.fif
2,102a,2.390371,-17.843979,-3.338771,7.536740,-10.998419,18.220947,16.625307,-4.156407,-17.935160,...,102,a,f,20,20;27,Nee,Control,F:\Stage\ePODIUM\Data\ePodium (Projectfolder)\...,F:\Stage\ePODIUM\ePodium\dyslexiaprediction_na...,102a_epo.fif
3,102b,7.074118,23.187111,-9.459183,25.835532,18.345802,34.434200,34.885866,22.527005,37.353435,...,102,b,f,23,23;16,Nee,Control,F:\Stage\ePODIUM\Data\ePodium (Projectfolder)\...,F:\Stage\ePODIUM\ePodium\dyslexiaprediction_na...,102b_epo.fif
4,103a,-18.065883,-17.785199,-12.858838,-8.709094,9.764843,-1.953743,-2.150446,11.090374,0.289517,...,103,a,f,20,20;23,m,At risk,F:\Stage\ePODIUM\Data\ePodium (Projectfolder)\...,F:\Stage\ePODIUM\ePodium\dyslexiaprediction_na...,103a_epo.fif


In [11]:
df.columns

Index(['eeg_file', '2_Fp1', '2_AF3', '2_F7', '2_F3', '2_FC1', '2_FC5', '2_T7',
       '2_C3', '2_CP1',
       ...
       'ParticipantID', 'test', 'sex', 'age_months', 'age_months_days',
       'dyslexic_parent', 'Group_AccToParents', 'eeg_path', 'path_epoch',
       'epoch_file'],
      dtype='object', length=139)

In [12]:
df = df.drop(['eeg_file','age_months_days',
       'dyslexic_parent', 'eeg_path', 'path_epoch',
       'epoch_file'], axis =1)

In [13]:
df.to_csv('df_avg_mmr.csv', index=False)